In [ ]:
## Streamlined methods to prepare weight categories for a Proportional Area Analysis

# Author: Kaitlin Lubetkin edited by Alex Traynor
# Created: 9/24/21
# Last Edited: 8/03/23
                                                                                                                                                                                            

In [2]:
# Setup packages
import os
import arcpy

## Prepare workspace and define locations (make sure to change file paths and layer names to your own)
map = arcpy.mp.ArcGISProject("CURRENT").listMaps("Weighted Analysis")[0]

# Parent/root folder for analysis
parent_folder = r'C:\Users\alaurencetraynor\Documents\OR\ValeDO'

gdb = parent_folder + '/ValeDO.gdb'

# reporting unit
reporting_unit = gdb + r'\CowLakesAllotmentGroup_dissolve2_SummarizeWithin'

# sample design polys
sdd_polys = parent_folder + r'\CLAA_Weighted_Analysis_v2.gdb\CLAA_Weighted_Analysis_v2.gdb\CLAA_SampleFrame'

sdd_strata = parent_folder + r'\CLAA_Weighted_Analysis_v2.gdb\CLAA_Weighted_Analysis_v2.gdb\CLAA_Strata'


In [3]:
# there is one sample frame that is unstratifed (Malheur 2021 design) 
# so ill need to add this to the strata festure class before the union
unstratified_sample_frames = 'OR_MalheurFO_2021_LUP_SampleFrame_1'
where_clause = 'TerrestrialSampleFrameID = '+ "'"+ unstratified_sample_frames+ "'"

# append to strata fc
# need to use field mapping here since they have different fields
arcpy.management.Append(sdd_polys, sdd_strata, "NO_TEST", r'TerrestrialDesignStratumID "TerrestrialStrataumID" true true false 255 Text 0 0,First,#,sdd_polys,TerrestrialSampleFrameID,0,255', '', where_clause)

# clip strata to RU
sdd_strata_clip = arcpy.analysis.PairwiseClip(sdd_strata, reporting_unit, gdb + '/sdd_poly_clip', None)

In [12]:
# for the unstratified designs need to copy over sample frame id
clause =  "'TerrestrialSampleFrameID' is NULL"
print(clause)

with arcpy.da.UpdateCursor(sdd_strata_clip,["TerrestrialSampleFrameID","TerrestrialDesignStratumID"],where_clause = clause) as cursor:
    for row in cursor:
        row[0] = row[1]

'TerrestrialSampleFrameID' is NULL


In [24]:
# Split out each design
arcpy.analysis.SplitByAttributes(sdd_strata_clip, gdb, 'TerrestrialSampleFrameID')

<Result 'C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO\\ValeDO.gdb'>

In [31]:
# Union between all unique strata
# make a list of all sample frames fcs
with arcpy.da.SearchCursor(sdd_strata_clip,("TerrestrialSampleFrameID")) as cursor:
    # make empty list
    strata_list = list()
    for row in cursor:
        strata = gdb + "/" + row[0]
        strata_list.append(strata)

strata_list = list(set(strata_list))
print(strata_list)

# do union with list
strata_union = arcpy.Union_analysis(in_features = strata_list, out_feature_class = gdb + "/strata_union")

['C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO/ValeDO.gdb/OR_MalheurFO_2017_LUP_SampleFrame_1', 'C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO/ValeDO.gdb/OR_ValePAC_2021_LUP_SampleFrame_1', 'C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO/ValeDO.gdb/OR_SO_2016_PAC_SampleFrame_1', 'C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO/ValeDO.gdb/OR_MalheurFO_2021_LUP_SampleFrame_1']


In [32]:
strata_union = r'C:\Users\alaurencetraynor\Documents\OR\ValeDO\ValeDO.gdb\strata_union'

In [33]:
## Clean up the Unioned layer (can just run straight through this cell)

#Scratch geodatabase to be used to store intermediate layers
arcpy.management.CreateFileGDB(parent_folder, 'scratch.gdb')
scratch_gdb = parent_folder + '\\scratch.gdb'

#Just to be safe, repair geometry
arcpy.management.RepairGeometry(strata_union, "DELETE_NULL")

#Get all TerrestrialDesignStratumID field names
fields = arcpy.ListFields(strata_union)
strataIDFields = []
for f in fields:
    if f.name.startswith("TerrestrialDesignStratumID"):
        strataIDFields.append(f.name)
    
#Add and calculate a field for the combined strata
arcpy.management.AddField(strata_union, "strata_combo", "TEXT", field_length = 255)
addr = ["!" + idField + "!" for idField in strataIDFields] 
arcpy.management.CalculateField(strata_union, "strata_combo", 
                                "ConcatAddr(" + ','.join([a for a in addr]) + ")", 
                                "PYTHON3", 
                                "def ConcatAddr(*args): return ''.join([str(i) for i in args if i not in(None,' ')]) ", 
                                "TEXT", 
                                "NO_ENFORCE_DOMAINS")
print("strata combo populated")

#Add and calculate a field for the area in hectares
arcpy.management.AddField(strata_union, "area_hectares", "DOUBLE")
arcpy.management.CalculateGeometryAttributes(strata_union, 
                                             "area_hectares AREA_GEODESIC", 
                                             area_unit = "HECTARES")

#Save new, cleaned up copy
fm =('Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,' + strata_union + ',Shape_Length,-1,-1;' + 
     'Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,' + strata_union + ',Shape_Area,-1,-1;' + 
     'area_hectares "area_hectares" true true false 8 Double 0 0,First,#,' + strata_union + ',area_hectares,-1,-1;' + 
     'strata_combo "strata_combo" true true false 255 Text 0 0,First,#,' + strata_union + ',strata_combo,0,255;')

arcpy.conversion.FeatureClassToFeatureClass(strata_union, scratch_gdb, "CLAA_strata_UNION_clean", 
                                            field_mapping=fm)

strata combo populated


<Result 'C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO\\scratch.gdb\\CLAA_strata_UNION_clean'>

In [37]:
## Next, manually fix slivers (<2 ha) by giving them the strata_combo of an adjacent polygon
## let just delete the slivers instead?
with arcpy.da.UpdateCursor(scratch_gdb + "/CLAA_strata_UNION_clean", "area_hectares") as cursor:
    for row in cursor:
        if row[0] < 2:
            cursor.deleteRow()
# Then proceed with next Jupyter cell

In [38]:
## Dissolve the unioned and cleaned strata (can just run straight through this cell)

#Dissolve based on the "strata_combo"
arcpy.management.Dissolve("CLAA_strata_UNION_clean", 
                          gdb + "\\CLAA_strata_UNION_Dissolve", 
                          "strata_combo")
#Recalculate hectares
arcpy.management.CalculateGeometryAttributes("CLAA_strata_UNION_Dissolve", 
                                             "area_hectares AREA_GEODESIC", 
                                             area_unit = "HECTARES")
#Add weight category field
arcpy.management.CalculateField("CLAA_strata_UNION_Dissolve", 
                                "wgtcat", 
                                "!OBJECTID!", "PYTHON3",
                                field_type="SHORT")

<Result 'CLAA_strata_UNION_Dissolve'>

In [ ]:
## Double check small (1-2 ha) polygons
## If necessary, return to the UNION and reassign strata combo & sample design, then redo the Dissolve

In [50]:
# join wgtcat info to all the other inputs, namely sampled and rejected points
# grab the benchmarked points from agol and subset them to remove LMF and other designs we arent using
# We are also exlcuding points prior to 2017
points = 'https://services1.arcgis.com/KbxwQRRfWyEYLgp4/arcgis/rest/services/Cow_Lakes_Allotment_Group_AIM_Benchmark_Summary_WFL1/FeatureServer/0'
clause = "ProjectNam <> 'LMF' AND DateVisite >= timestamp '2017-01-01 00:00:00'"
arcpy.FeatureClassToFeatureClass_conversion(points, gdb,"benchmarked_points", clause)

# spatial join to wgtcats
# also we dont have any rejections whch is nice
arcpy.SpatialJoin_analysis(gdb+"/benchmarked_points", scratch_gdb + "/CLAA_strata_UNION_Dissolve", gdb+"/benchmarked_points_wgtcats")


<Result 'C:\\Users\\alaurencetraynor\\Documents\\OR\\ValeDO\\ValeDO.gdb\\benchmarked_points_wgtcats'>